# Instalacion de Requisitos y Creación de Spark Session

In [0]:
!pip install pyspark

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.tuning import CrossValidatorModel

In [0]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("KMeans-Caso-Real").getOrCreate()

# Ingesta y Procesamiento de Datos

In [0]:
# Carga de datos desde un archivo CSV
data = spark.read.option("header", "true").csv("/FileStore/practica-3/view_caso_real.csv")
data.toPandas().head()

In [0]:
# Correccion de nombres en la columnas.
data = data.withColumnRenamed('Cuontas_Restantes','Cuotas_Restantes')

# Rellenar valores nulos en la columna "Aclaracion" con un valor predeterminado (por ejemplo, "Sin Aclaracion")
data = data.withColumn("Aclaracion", when(data["Aclaracion"].isNull(), "Sin Aclaracion").otherwise(data["Aclaracion"]))

# Aplicar el StringIndexer al DataFrame con valores predeterminados
aclaracionIndexer = StringIndexer(inputCol="Aclaracion", outputCol="Aclaracion_index")
df_clean = aclaracionIndexer.fit(data).transform(data)

# Convertir las columnas seleccionadas de string a int
featuresCols = ["Monto_Moneda_Local", "Monto_Moneda_Extranjera", 
                "Cuotas_Pagadas", "Cuotas_Devengadas", 
                "Cuotas_Restantes", "Aclaracion_index"]
for feature in featuresCols:
    df_clean = df_clean.withColumn(feature, col(feature).cast("int"))
    
df_clean.toPandas().head()

#### Realizar un agrupamiento de las variables cualitativas que usara (tabla de frecuencia).

In [0]:
from pyspark.sql.functions import count

# Calcula la frecuencia de cada valor único en la columna "status"
status_frequencies = df_clean.groupBy("Aclaracion").agg(count("*").alias("frequency"))
# Muestra la tabla de frecuencias
status_frequencies.show()

#### Para el caso de una variable cuantitativa usar el método describe para obtener estadísticos descriptivos.

In [0]:
cuantitativas = ["Monto_Moneda_Local", "Monto_Moneda_Extranjera", 
                "Cuotas_Pagadas", "Cuotas_Devengadas", 
                "Cuotas_Restantes"]

# Usa el método "describe" en estas columnas.
describe_stats = df_clean.select(*cuantitativas).describe()

# Muestra las estadísticas descriptivas.
describe_stats.show()

# División de los datos

In [0]:
assembler = VectorAssembler(inputCols=featuresCols, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)

# Paso 4: División de datos en conjuntos de entrenamiento y prueba
train_data, test_data = df_clean.randomSplit([0.7, 0.3], seed=42)  # 70% para entrenamiento, 30% para prueba


# Creación y Entrenamiento del Modelo

In [0]:
# Paso 5: Definir el modelo K-Means
kmeans = KMeans().setSeed(1).setFeaturesCol("scaledFeatures").setPredictionCol("cluster")

# Paso 6: Crear un Pipeline
pipeline = Pipeline(stages=[assembler, scaler, kmeans])

# Paso 7: Crear una cuadrícula de parámetros para búsqueda en cuadrícula
param_grid = ParamGridBuilder().addGrid(kmeans.k, [3, 4, 5, 6, 7, 8, 2])\
.addGrid(kmeans.initMode, ["k-means||", "random"])\
.addGrid(kmeans.maxIter, [10, 20, 30, 40,45,50])\
.build()

# Paso 8: Configurar el evaluador
evaluator = ClusteringEvaluator(predictionCol="cluster", featuresCol="features")

# Paso 9: Configurar la validación cruzada
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Paso 10: Ajustar el modelo con búsqueda en cuadrícula
cv_model = crossval.fit(train_data)

# Evaluación del Modelo

In [0]:
# Obtener el mejor modelo
best_model = cv_model.bestModel

# Aplicar el mejor modelo a los datos de prueba
predictions = best_model.transform(test_data)

# Mostrar la configuracion de hiperparámetros del mejor modelo
print(f"Mejor valor de K: {best_model.stages[-1].getK()}")
print(f"Mejor valor de initMode: {best_model.stages[-1].getInitMode()}")
print(f"Mejor valor de maxIter: {best_model.stages[-1].getMaxIter()}")

# Evaluacion del modelo utilizando Silhouette
evaluator = ClusteringEvaluator(predictionCol="cluster", featuresCol="features", metricName="silhouette")
silhouette_score = evaluator.evaluate(predictions)


print(f"El Silhouette Score es: {silhouette_score}")

# Interpretación de los Grupos

In [0]:
# Obtener los centros de los clusters
centers = best_model.stages[2].clusterCenters()
print("Cluster Centers:")
for center in centers:
    print(center)
# Graficar los clusters
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# Aplicar PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
centers_2d = pca.fit_transform(centers)

# Crear una figura
plt.figure(figsize=(8, 6))

# Dibujar los puntos de los centros de los clusters en 2D
plt.scatter(centers_2d[:, 0], centers_2d[:, 1], c='red', marker='x', s=100, label='Centers')

# Puedes etiquetar los puntos con los números de los clusters
for i, center in enumerate(centers):
    plt.annotate(str(i), (centers_2d[i, 0], centers_2d[i, 1]), fontsize=12)

plt.xlabel('PCA Dimension 1')
plt.ylabel('PCA Dimension 2')
plt.title('Centros de los Clusters en 2D')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

In [0]:
cluster_counts = predictions.groupBy("cluster").count()
# Muestra los resultados
cluster_counts.show()

In [0]:
from pyspark.sql.functions import mean, round
# Agrupa los datos por la columna de predicción
grouped_data = predictions.groupBy("cluster")

# Calcula estadísticas descriptivas para cada grupo y característica
mean_data = grouped_data.agg(
    round(mean("Monto_Moneda_Local"), 2).alias("Monto_Moneda_Local"),
    round(mean("Monto_Moneda_Extranjera"), 2).alias("Monto_Moneda_Extranjera"),
    round(mean("Cuotas_Pagadas"), 2).alias("Cuotas_Pagadas"),
    round(mean("Cuotas_Devengadas"), 2).alias("Cuotas_Devengadas"),
    round(mean("Cuotas_Restantes"), 2).alias("Cuotas_Restantes"),
).toPandas()

mean_data.head()

In [0]:
from pyspark.sql.functions import countDistinct,collect_set

# Agrupa los datos por la columna "cluster" y cuenta valores únicos de "aclaración"
count_data = predictions.groupBy("cluster").agg(
    countDistinct("aclaracion").alias("count_aclaracion"),
    collect_set("aclaracion").alias("unique_aclaracion_values")
)

# Muestra los resultados
count_data.show(truncate=False)